<a href="https://colab.research.google.com/github/nug1209/PwC_Switzerland_Digital_Intelligence_Virtual_Case_Experience/blob/main/PwC_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this task we deal with merchant loan data. We will create a forecast about how much will the total repayment will be after these loans are finished (after 30 months since the loans started). The formula for counting the repayment is provided. We will also create percentage values for each repayment in comparison with the origination amount.

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import datetime

In [99]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Virtual Experience/PwC_Task_2_Data.csv'

df = pd.read_csv(path, delimiter=';')
df_copy = df.copy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
df

,Unnamed: 0,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,...,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,...,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
1,30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,...,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
2,31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,...,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
3,31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,...,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
4,30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,...,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05
5,31.10.2019,15652952.20,0.00,0.00,0.00,0.00,0.00,2289453.76,4682354.31,1659503.89,...,763523.36,742787.97,558085.95,461806.22,358671.23,281881.11,241719.91,182730.05,144953.58,119260.10
6,30.11.2019,15107713.30,0.00,0.00,0.00,0.00,0.00,0.00,2162283.09,4637701.69,...,930720.35,697500.94,667277.73,547749.09,387987.02,309448.86,283876.04,215635.85,185516.45,141560.57
7,31.12.2019,17004745.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2402403.37,...,1245452.39,1116505.94,803590.21,724956.28,545397.33,458832.95,393971.01,333818.53,286831.44,216447.57
8,31.01.2020,16794379.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1505493.21,1182983.95,955821.48,846061.73,683116.81,560572.44,468203.76,349067.92,309854.99,267813.78
9,29.02.2020,19217205.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,6142911.08,1833677.81,1317065.75,1108494.37,918465.33,719913.69,587381.96,498801.31,371563.77,294941.22


In [101]:
df = df.rename(columns={'Unnamed: 0':'origination_date', 'Origination Amount':'origination_amount'})
# df.head(3)

In [102]:
df.shape

(20, 22)

In [103]:
to_remove = ['origination_date', 'origination_amount']
value_columns = [i for i in list(df.columns) if i not in to_remove]

df = pd.melt(df, id_vars=['origination_date', 'origination_amount'], value_vars=value_columns)
df = df.rename(columns={'variable':'repayment_date', 'value':'repayment_amount'})

In [104]:
df = df[df['repayment_amount'] != 0.00]
# df

In [105]:
df['origination_date'] = pd.to_datetime(df['origination_date'], format='%d.%m.%Y')
df['repayment_date'] = pd.to_datetime(df['repayment_date'], format='%d.%m.%Y')
# df.info()

In [106]:
df = df.sort_values(by=['origination_date', 'repayment_date'])
df

,origination_date,origination_amount,repayment_date,repayment_amount
0,2019-05-31,10018746.17,2019-05-31,1443069.08
20,2019-05-31,10018746.17,2019-06-30,3332200.33
40,2019-05-31,10018746.17,2019-07-31,1328138.75
60,2019-05-31,10018746.17,2019-08-31,928085.74
80,2019-05-31,10018746.17,2019-09-30,736418.27
...,...,...,...,...
377,2020-10-31,27699586.46,2020-11-30,8752706.73
397,2020-10-31,27699586.46,2020-12-31,1503544.68
378,2020-11-30,29872889.68,2020-11-30,4383982.78
398,2020-11-30,29872889.68,2020-12-31,8383025.07


In [107]:
# for i in value_columns:
#   df[i + '_actual_repayment_percent'] = 100 * df[i] / df['origination_amount']

# df.columns

In [108]:
sum([1, 2, 3])

6

In [109]:
# p = [p1, p2]

# for i in np.arange(31):

# pi = max(aa, 0)

# aa = p2 * np.log(bb)

# bb = 1 + (cc*dd)

# cc = 1 - ee

# dd = (i - 1) / 30

# ee = 1 - ff

# ff = sum(p)
	

In [110]:
np.arange(31)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])